## Overview
This notebook performs automated sentiment analysis and categorization of restaurant reviews using Google's Gemini AI model. The analysis categorizes each review into specific aspects of the restaurant experience:

- Hizmet (Service)
- Tat (Taste)
- Ortam (Ambiance)
- Fiyat-Performans (Price-Performance)
- Menü Çeşitliliği (Menu Variety)
- Temizlik (Cleanliness)

## How it works
1. The script reads restaurant reviews from a CSV file
2. Each review is processed through the Gemini 2.5 Flash model
3. The AI model analyzes the text and assigns sentiment scores (-1 to 1) for relevant categories
4. Results are saved in JSON format for each review
5. The process runs in batches of 500 reviews to manage memory and API usage efficiently

## Technical Details
- Uses Google's `google.generativeai` library
- Processes reviews in batches of 500
- Includes error handling and progress saving
- Output is stored in CSV format with the original reviews and AI analysis results

In [1]:
pip install -q -U google-genai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import google.generativeai as genai
import pandas as pd
import math
import time

# Configure the API key
api_key = "X"  
genai.configure(api_key=api_key)

SOURCE_FILE = "/Users/Serra/Desktop/bitirme/fourth_10k.csv"
SAVE_PATH = "/Users/Serra/Desktop/bitirme/updated_fourth_10k.csv"
BUFFER_SIZE = 500
PAUSE_SECONDS = 1

data = pd.read_csv(SOURCE_FILE, delimiter=",")
if "AI_Result" not in data.columns:
    data["AI_Result"] = None



# Create the model

model = genai.GenerativeModel(
    "gemini-2.5-flash",
    system_instruction="""
    Sana bazı yorumlar vereceğim.
    Bu yorumları "Hizmet", "Tat", "Ortam", "Fiyat-Performans", "Menü Çeşitliliği", "Temizlik"
    kategorilerinden hangilerine uyuyorsa ona göre sınıflandır.
    Çıktıyı JSON formatında ver. Örnek:
    {
        "categories": {
            "Tat": null,
            "Hizmet": -0.3,
            "Ortam": 0.8,
            "Fiyat-Performans": null,
            "Menü Çeşitliliği": null,
            "Temizlik": -0.7
        }
    }
    """
)

# ==== PROCESS DATA ====
yorumlar = list(data["Yorum"])
total_chunks = math.ceil(len(yorumlar) / BUFFER_SIZE)

for chunk_idx in range(total_chunks):
    start = chunk_idx * BUFFER_SIZE
    end = start + BUFFER_SIZE
    batch = yorumlar[start:end]

    print(f"Processing chunk {chunk_idx+1}/{total_chunks} ({len(batch)} reviews)...")

    for idx, yorum in enumerate(batch, start=start):
        try:
            response = model.generate_content(
                yorum,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.4,
                    response_mime_type="application/json"
                )
            )
            result_text = response.text.strip()
        except Exception as e:
            print(f"Error on review index {idx}: {yorum}\n{e}")
            result_text = None

        data.loc[idx, "AI_Result"] = result_text
        print(f"Yorum: {yorum}")
        print(f"Cevap: {result_text}")
        print("-" * 50)

    # Save after each chunk
    data.to_csv(SAVE_PATH, index=False)
    print(f"💾 Progress saved to {SAVE_PATH}")
    time.sleep(PAUSE_SECONDS)

print("✅ Finished processing all reviews.")

Processing chunk 1/20 (500 reviews)...
Yorum: Servis geç geldi tabağımdaki ürün eksikti hizmet düşük
Cevap: {
    "categories": {
        "Tat": null,
        "Hizmet": -0.8,
        "Ortam": null,
        "Fiyat-Performans": null,
        "Menü Çeşitliliği": null,
        "Temizlik": null
    }
}
--------------------------------------------------
Yorum: Çok güzel bir yer!
Cevap: {
    "categories": {
        "Tat": null,
        "Hizmet": null,
        "Ortam": 0.8,
        "Fiyat-Performans": null,
        "Menü Çeşitliliği": null,
        "Temizlik": null
    }
}
--------------------------------------------------
Yorum: Dondurmanın koyulduğu kaplar plastiklerde hep kırıklar olmuş ve o plastik de bize geldi borcumuz da kaçsaydı ya hizmet
Cevap: {
    "categories": {
        "Tat": null,
        "Hizmet": -0.9,
        "Ortam": null,
        "Fiyat-Performans": null,
        "Menü Çeşitliliği": null,
        "Temizlik": -0.7
    }
}
--------------------------------------------------
Y